#### All the required dependencies

In [71]:
import numpy as np

#### Building a class for a Perceptron/Neuron

In [72]:
class Perceptron:
    def __init__(self, weights, bias):
        self.weights = np.array(weights)
        self.bias = bias

    def predict(self, inputs):
        inputs = np.array(inputs)
        activation_sum = np.dot(inputs, self.weights.T) + self.bias
        return self.activation_function(activation_sum)

    def activation_function(self, value):
        if(value>0):
            return 1
        else:
            return 0

#### Instantiating an object of Perceptron class to replicate the Digital Logic AND gate

In [73]:
AND_gate = Perceptron(weights=[1,1], bias=-1.5)
#### Testing the AND_gate with input values
valid_inputs = [[0,0],
                [1,0],
                [0,1],
                [1,1]]
for input in valid_inputs:
    print(f'Input: {input}\t Output: {AND_gate.predict(input)}')

Input: [0, 0]	 Output: 0
Input: [1, 0]	 Output: 0
Input: [0, 1]	 Output: 0
Input: [1, 1]	 Output: 1


### Proof that we can't build a XOR gate with a single neuron that has a linear activation function
XOR gate: <br>
[0,0] -> 0 <br>
[1,0] -> 1 <br>
[0,1] -> 1 <br>
[1,1] -> 0 <br>
WLOG, we can assume the perceptron gets activated/returns 1 when the weighted sum is greater than 0, and 0 otherwise.
Let the weights be [w<sub>1</sub> ,w<sub>2</sub>] and the bias be *b*. <br>
Constructing 4 equations out of the input-output pairs: <br>
*b* <= 0 (for [0,0] -> 0) <br>
*b* + w<sub>1</sub> > 0 (for [1,0] -> 1) <br>
*b* + w<sub>2</sub> > 0 (for [0,1] -> 1) <br>
*b* + w<sub>1</sub> + w<sub>2</sub> <= 0 (for [1,1] -> 0) <br>

It can be observed that these 4 equations are inconsistent, hence we can't assign the weights and biases of a single perceptron to replicate a XOR gate.

### Building a XOR gate using a hidden layer

In [74]:
hidden_neuron_1 = Perceptron(weights = [1,1], bias = -0.5)
hidden_neuron_2 = Perceptron(weights = [-1,-1], bias = 1.5)
output_neuron = Perceptron(weights = [1,1], bias = -1.5)

for input in valid_inputs:
    hidden_1_output = hidden_neuron_1.predict(input)
    hidden_2_output = hidden_neuron_2.predict(input)
    xor_gate_output = output_neuron.predict([hidden_1_output, hidden_2_output])
    print(f'Input: {input}\t Output: {xor_gate_output}')

Input: [0, 0]	 Output: 0
Input: [1, 0]	 Output: 1
Input: [0, 1]	 Output: 1
Input: [1, 1]	 Output: 0


#### Building a multi-layer Perceptron, with one hidden layer and a final layer

In [75]:
class Two_Layer_Perceptron:
    def __init__(self, hidden_layer_weights, hidden_layer_bias, output_layer_weights, output_layer_bias):
        self.hidden_layer_weights = np.array(hidden_layer_weights)
        self.hidden_layer_bias = np.array(hidden_layer_bias)
        self.output_layer_weights = np.array(output_layer_weights)
        self.output_layer_bias = output_layer_bias

    def predict(self, inputs):
        inputs = np.array(inputs)
        hidden_layer_output = self.activation_function(np.dot(inputs, self.hidden_layer_weights.T) + self.hidden_layer_bias)
        output_layer_output = self.activation_function(np.dot(hidden_layer_output, self.output_layer_weights.T) + self.output_layer_bias)
        return output_layer_output
    
    def activation_function(self, value):
        return value > 0


#### Employing XOR gate in this new class

In [76]:
XOR_gate= Two_Layer_Perceptron(hidden_layer_weights=[[1,1],
                                                     [-1,-1]],
                                hidden_layer_bias=[-0.5,1.5],
                                output_layer_weights=[1,1],
                                output_layer_bias=-1.5)
for input in valid_inputs:
    print(f'Input: {input}\t Output: {int(XOR_gate.predict(input))}')

Input: [0, 0]	 Output: 0
Input: [1, 0]	 Output: 1
Input: [0, 1]	 Output: 1
Input: [1, 1]	 Output: 0


#### Building a Full Adder using this new multi-layer Perceptron

In [77]:
class FullAdder:
    def __init__(self):
        self.AND_gate = Perceptron(weights=[1,1], bias=-1.5)
        self.OR_gate = Perceptron(weights=[1,1], bias=0)
        self.XOR_gate = Two_Layer_Perceptron(hidden_layer_weights=[[1,1],
                                                     [-1,-1]],
                                hidden_layer_bias=[-0.5,1.5],
                                output_layer_weights=[1,1],
                                output_layer_bias=-1.5)
        
    def add(self, a, b, carry_in):
        xor_a_b = self.XOR_gate.predict([a,b])
        sum = self.XOR_gate.predict([xor_a_b,carry_in])
        # sum=self.XOR_gate.predict([xor_a_b,xor_carry_a_b])

        and_a_b = self.AND_gate.predict([a,b])
        and_carry_xor_a_b = self.AND_gate.predict([carry_in, xor_a_b])
        carry_out = self.OR_gate.predict([and_a_b, and_carry_xor_a_b])

        return (sum, carry_out)

In [78]:
for a in range(0,2):
    for b in range(0,2):
        for carry_in in range(0,2):
            print(f'A: {a}, B: {b}, Carry_in: {carry_in}, Sum: {int(FullAdder().add(a,b,carry_in)[0])}, Carry_out: {int(FullAdder().add(a,b,carry_in)[1])}')

A: 0, B: 0, Carry_in: 0, Sum: 0, Carry_out: 0
A: 0, B: 0, Carry_in: 1, Sum: 1, Carry_out: 0
A: 0, B: 1, Carry_in: 0, Sum: 1, Carry_out: 0
A: 0, B: 1, Carry_in: 1, Sum: 0, Carry_out: 1
A: 1, B: 0, Carry_in: 0, Sum: 1, Carry_out: 0
A: 1, B: 0, Carry_in: 1, Sum: 0, Carry_out: 1
A: 1, B: 1, Carry_in: 0, Sum: 0, Carry_out: 1
A: 1, B: 1, Carry_in: 1, Sum: 1, Carry_out: 1


#### Implementing a Ripple Carry Adder

In [80]:
a=547434634
b=83432

def binary_representation(number):
    binary_string = bin(number)[2:]
    if len(binary_string) < 64:
        binary_string = '0' * (64 - len(binary_string)) + binary_string
    return binary_string


def ripple_carry_adder(a, b):
    binary_a = binary_representation(a)
    binary_b = binary_representation(b)

    carry = 0
    sum = 0
    sum_binary = ''

    for i in range(63, -1, -1):
        bit_a = int(binary_a[i])
        bit_b = int(binary_b[i])

        sum, carry = FullAdder().add(bit_a, bit_b, carry)
        sum=int(sum)
        carry=int(carry)
        sum_binary = str(sum) + sum_binary

    sum_binary = str(carry) + sum_binary

    return int(sum_binary, 2)

print(f'Sum: {ripple_carry_adder(a, b)}')

Sum: 547518066
